<a href="https://colab.research.google.com/github/anilbhatt1/Skunkworks_Mediapipe_Jitter/blob/main/Mediapipe_pose_stabilize_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Usage example of MediaPipe Pose Solution API in Python (see also http://solutions.mediapipe.dev/pose).

In [ ]:
# Minimum dependency for MediaPipe Solutions Python API is opencv-python
!pip install opencv-python~=3.4.11
!pip install mediapipe

     |████████████████████████████████| 49.1MB 35kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 34.5MB 1.3MB/s 


In [ ]:
lm_dict = {0:'nose',
           1:'right_eye_inner', 2:'right_eye', 3:'right_eye_outer',
           4:'left_eye_inner', 5:'left_eye', 6:'left_eye_outer',
           7:'right_ear', 8:'left_ear',
           9:'right_mouth',10:'left_mouth',
           11:'right_shoulder', 12:'left:shoulder',
           13:'right_elbow', 14:'left_elbow',
           15:'right_wrist', 16:'left_wrist',
           17: 'right_pinky', 18:'left_pinky',
           19:'right_index', 20:'left_index',
           21:'right_thumb', 22:'left_thumb',
           23:'right_hip', 24:'left_hip'}

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose    = mp.solutions.pose

pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture('/content/9secs_avi.avi')
#cap = cv2.VideoCapture('/content/4secs_avi.avi')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/content/9secs_pose_v10_0.04.avi',fourcc, 20.0, (352,640))
#out = cv2.VideoWriter('/content/4secs_pose_v10_0.00.avi',fourcc, 20.0, (1080,1920))

frame_cnt, delta_x, delta_y = 0, 0, 0
prev_lst =  []
lst = [0,0]
for i in range(25):
    prev_lst.append(lst) 

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame_cnt += 1
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)        
      
        if frame_cnt == 1:
            for i in range(25):
                lst = [results.pose_landmarks.landmark[i].x, results.pose_landmarks.landmark[i].y]
                prev_lst[i] = lst
        else:
            for i in range(25):
                x_curr, y_curr  = results.pose_landmarks.landmark[i].x, results.pose_landmarks.landmark[i].y
                delta_x = abs(x_curr - prev_lst[i][0]) / prev_lst[i][0]
                delta_y = abs(y_curr - prev_lst[i][1]) / prev_lst[i][1]
                
                if delta_x < 0.04:
                    results.pose_landmarks.landmark[i].x  = prev_lst[i][0]  
                else:
                    print('---delta_x > 0.04', 'lm#', lm_dict[i], '#', frame_cnt, 'delta_x:', delta_x)                   
                if delta_y < 0.04:
                    results.pose_landmarks.landmark[i].y  = prev_lst[i][1]
                else:                
                    print('---delta_y > 0.04', 'lm#', lm_dict[i], '#', frame_cnt, 'delta_y:', delta_y) 
                
                lst = [results.pose_landmarks.landmark[i].x, results.pose_landmarks.landmark[i].y]
                prev_lst[i] = lst

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        out.write(image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        print('End of Frame # :', frame_cnt)
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

---delta_x > 0.04 lm# left_wrist # 3 delta_x: 0.04852341667363119
---delta_x > 0.04 lm# left_pinky # 3 delta_x: 0.041054593623215306
---delta_x > 0.04 lm# left_index # 3 delta_x: 0.05804061483505954
---delta_x > 0.04 lm# left_thumb # 3 delta_x: 0.052625990702404424
---delta_x > 0.04 lm# left_elbow # 4 delta_x: 0.0447543669369108
---delta_x > 0.04 lm# right_pinky # 4 delta_x: 0.04022199940379466
---delta_x > 0.04 lm# right_index # 4 delta_x: 0.04304775463862888
---delta_x > 0.04 lm# left_index # 4 delta_x: 0.04453793744312661
---delta_x > 0.04 lm# right_thumb # 4 delta_x: 0.04310512080928346
---delta_x > 0.04 lm# left_wrist # 5 delta_x: 0.049490733318296766
---delta_x > 0.04 lm# left_pinky # 5 delta_x: 0.07198490065959219
---delta_x > 0.04 lm# left_thumb # 5 delta_x: 0.06413987989030769
---delta_x > 0.04 lm# left_elbow # 6 delta_x: 0.05312367709156068
---delta_x > 0.04 lm# right_wrist # 6 delta_x: 0.04109646032381785
---delta_x > 0.04 lm# left_index # 6 delta_x: 0.05862207043533688
---d

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose    = mp.solutions.pose

pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture('/content/4secs_avi.avi')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/content/4secs_pose_no_thresh.avi',fourcc, 20.0, (1080,1920))

frame_cnt = 0

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame_cnt += 1
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)        
      
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        out.write(image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        print('End of Frame # :', frame_cnt)
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

End of Frame # : 163
